# TFF/Anthos Hello World

Before running the notebook make sure that you have obtained access credentials to GCP by executing 

```
gcloud auth login
```

from the JupyterLab terminal

In [1]:
import collections
import time
import numpy as np
import grpc
import sys

import tensorflow as tf
import tensorflow_federated as tff

import nest_asyncio
nest_asyncio.apply()

### Define federated computation

Note that we need the explicit `tf.function` to make `tf.print` work.

In [2]:
@tff.tf_computation(tf.int64)
@tf.function
def hello(n):
    tf.print("Hello: ", n)
    return n

@tff.federated_computation(tff.FederatedType(tf.int64, tff.CLIENTS))
def compute_federated_sum(federated_n):
    return tff.federated_sum(tff.federated_map(hello, federated_n))

### Run computation locally

In [35]:
tff.backends.native.set_local_execution_context()

In [36]:
compute_federated_sum([1, 2, 1])

Hello:  2
Hello:  1
Hello:  1


4

### Run computation on the executor in the `server` cluster

In [37]:
port = 8000
ip_address = '10.108.9.176' # Cluster IP

channels = [grpc.insecure_channel(f'{ip_address}:{port}')]
tff.backends.native.set_remote_execution_context(channels, rpc_mode='REQUEST_REPLY')

The first call may hang. Interrupt and execute it again.

In [41]:
compute_federated_sum([1])

1

#### Double check that the computation executed on the remote executor in the server cluster

Get cluster credentials.

In [42]:
server_cluster_name = 'tff-server'
zone = 'us-west1-a'

!gcloud container clusters get-credentials {server_cluster_name} --zone {zone}

Fetching cluster endpoint and auth data.
kubeconfig entry generated for tff-server.


List pods

In [43]:
namespace = 'tff'

pods = !kubectl get pods -n {namespace} --no-headers -o custom-columns=":metadata.name"
remote_executor_pod = [pod for pod in pods if pod.startswith('remote')][0]
remote_executor_pod

'remote-executor-7944fc4874-qgfrs'

Retrieve logs

In [44]:
logs = !kubectl logs {remote_executor_pod}
logs[-3:-1]

['Hello:  1', 'Hello:  1']

### Run computation on the client executors

In [45]:
port = 8000
ip_addresses = ['35.233.216.138', '34.82.15.47'] # LoadBalancer IPs

channels = [grpc.insecure_channel(f'{ip_address}:{port}') for ip_address in ip_addresses]
tff.backends.native.set_remote_execution_context(channels, rpc_mode='REQUEST_REPLY')

The first call may hang. Interrupt and execute it again.

In [56]:
compute_federated_sum([1, 2])

3

#### Double check that the computation executed on the remote executor on one of the client cluster

Get cluster credentials.

In [57]:
client_cluster_name = 'tff-client-1'
zone = 'us-west1-a'

!gcloud container clusters get-credentials {client_cluster_name} --zone {zone}

Fetching cluster endpoint and auth data.
kubeconfig entry generated for tff-client-1.


List pods

In [58]:
namespace = 'tff'

pods = !kubectl get pods -n {namespace} --no-headers -o custom-columns=":metadata.name"
remote_executor_pod = [pod for pod in pods if pod.startswith('remote')][0]
remote_executor_pod

'remote-executor-7944fc4874-fbcv4'

Retrieve logs

In [59]:
logs = !kubectl logs {remote_executor_pod}
logs[-3:-1]

['Hello:  2', 'Hello:  2']